# DART Jovian Moons Movie Stabilization using OpenCV using Open Source Data.

An ROI centered movie of the Jovian Moons observations made by the NASA Double Asteroid Redirection Test (DART) spacecraft made on 2022-214. 
The data set is public and can be accessed in PDS here: https://pdssbn.astro.umd.edu/holdings/pds4-dart-v2.0/document_draco/SUPPORT/dataset.shtml 

### Data Processing
The data is downloaded from the NASA PDS archives as a set of PNGs using `wget`.
The data in the archive includes a time period after the Jovian Moons imaging period. During the last frames in the data collection the spacecraft continued imaging, but was no longer tracking Jupiter, we exclude the last images from our processed data set.
The data also shows a momentary period during which Jupiter (our subject) exits the frame and comes back. This happens, roughly, frames `dart_0397159786_25152_01_rad.png` through `dart_0397159918_20758_01_rad.png`. In order to keep the tracker from losing the subject in the image tracking portion of the software, we abandon those images in the ROI centered video result produced by the end of this Jupyter Notebook. 

At this point DART was approximately 16 million miles (26 million km) from Earth with Jupiter approximately 435 million miles (700 million km) away from the spacecraft.
The video shows from left to right are Ganymede, Jupiter, Europa, Io, and Callisto.

The total time of the data processed: 
- First frame: dart_0397157693_12079_01_rad.fits (MET='397157693'/ [sec] MET first pixel arrives at SBC)
- Last frame: dart_0397161169_08665_01_rad.fits (MET='397161169'/ [sec] MET first pixel arrives at SBC)
- Total time = 3476secs = 57.93mins = 57mins 55.8sec


### Subject (Jupiter) Tracking 

The code's main purpose is to track a subject within a video, ensuring that the subject remains centered in each frame of the output video, while also checking that the subject's ROI remains within the frame boundaries to avoid out-of-frame subjects. Adjustments and further optimizations can be made to suit specific tracking needs and video content.

- It reads a video file specified by the path using the OpenCV library.
- It allows the user to select an initial region of interest (ROI) by clicking and dragging the mouse on the first frame of the video.
- It initializes the OpenCV TrackerCSRT object to track the selected ROI as it moves within subsequent frames of the video.
- It sets up an output video file using the XVID codec and initializes a VideoWriter object to save the stabilized video.
- It processes the video frame by frame in a loop. For each frame:
  
    - It updates the tracker with the current frame and the tracked ROI's new position.
    - It calculates translation offsets to keep the subject within the frame centered at all times.
    - It checks if the subject's ROI is within the frame boundaries. If not, the frame is skipped.
    - If the subject's ROI is within the frame, it applies a translation to the entire frame to center the subject and draws the bounding box around the subject at the new position.
    - It writes the processed frame into the output video.
    - It displays the processed frame with the centered subject.
    - It checks for the 'q' key press to exit the loop.
      
After processing all frames, it releases the capture and video writer resources and closes any OpenCV windows.

If the subject falls out of frame for more than 2 images, tracking fails. 


In [1]:
import requests
from tqdm import tqdm
import cv2
import os
import matplotlib.pyplot as plt

/Users/rodrilm2/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Image Processing (PNG)

We will need to maintain the natural sort key order of our images to ensure the video is made using the right image order. 

In [2]:
import re

def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [
        int(text)
        if text.isdigit() else text.lower()
        for text in _nsre.split(s)]

#### Convert PNGs into AVI file using OpenCV

In [3]:
image_folder = '/Users/rodrilm2/jupyter/dart_jovian/pdssbn.astro.umd.edu/holdings/pds4-dart-v2.0/data_dracocal/cruise/2022/214_skipped_frames/'
video_name = 'dart_jovian_skipped_lapses.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
sorted_images = sorted(images, key=natural_sort_key)
frame = cv2.imread(os.path.join(image_folder, sorted_images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in tqdm(sorted_images):
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3475/3475 [00:14<00:00, 239.80it/s]


#### Convert AVI to mp4 so that we can see it on MacOS

In [4]:
def convert_avi_to_mp4(avi_file_path, output_name):
    os.popen("/opt/homebrew/bin/ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
    return True

In [5]:
dart_jovian_mp4="dart_jovian_skipped_lapses"
dart_jovian_centered_subject='dart_jovian_centered_subject'
convert_avi_to_mp4(video_name, dart_jovian_mp4)

True

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --di

#### Center ROI the Movie

In [9]:
import cv2
import numpy as np

# Read the video
cap = cv2.VideoCapture(dart_jovian_mp4+".mp4")

# Read the first frame
ret, frame = cap.read()
if not ret:
    exit()

# Select the ROI for tracking
roi = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)

# Initialize the tracker
tracker = cv2.TrackerCSRT_create()
tracker.init(frame, roi)

# Get the frame dimensions
height, width, _ = frame.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(dart_jovian_centered_subject+".mp4", fourcc, 1.0, (width, height))

# Process the video frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update the tracker
    success, roi = tracker.update(frame)

    # If tracking is successful and the subject ROI is within the frame boundaries
    if success:
        (x, y, w, h) = [int(i) for i in roi]

        # Calculate the translation offsets
        center_x, center_y = width // 2, height // 2
        offset_x, offset_y = center_x - (x + w // 2), center_y - (y + h // 2)

        # Check if the subject ROI is within the frame boundaries
        if 0 <= x + offset_x <= width - w and 0 <= y + offset_y <= height - h:
            # Apply the translation to the entire frame
            translation_matrix = np.float32([[1, 0, offset_x], [0, 1, offset_y]])
            frame = cv2.warpAffine(frame, translation_matrix, (width, height))

            # Draw the bounding box at the new position
            x, y = x + offset_x, y + offset_y

            # Write the frame into the output video
            out.write(frame)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and video writer, and destroy any OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()
print("DONE!")

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


DONE!
